In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tqdm.auto import tqdm

from juddges.llm_as_judge.data_model import PredictionLoader
from juddges.llm_as_judge.result_loading import (
    llm_as_judge_avg_scores,
    ngram_avg_scores,
)

In [3]:
res_dirs = [
    "data/experiments/predict/raw_vllm/pl_court_personal_rights/qwen_3_32b/info_extraction_annotated_json_refined/personal_rights/seed_42/",
    "data/experiments/predict/raw_vllm/pl_court_personal_rights/llama_3.1_8b_instruct/info_extraction_annotated_json_refined/personal_rights/seed_42/",
]


judge_resutls = {}
ngram_results = {}
for rdir in tqdm(res_dirs):
    pred_loader = PredictionLoader(root_dir=rdir, judge_name="gpt-4.1-mini")
    preds = pred_loader.load_predictions(verbose=True)
    res_judge = llm_as_judge_avg_scores(pred_loader)
    res_ngram = ngram_avg_scores(pred_loader)
    judge_resutls[pred_loader.config.llm.name] = res_judge
    ngram_results[pred_loader.config.llm.name] = res_ngram

  0%|          | 0/2 [00:00<?, ?it/s]

Loading predictions:   0%|          | 0/1811 [00:00<?, ?it/s]

Loading predictions:   0%|          | 0/1811 [00:00<?, ?it/s]

In [4]:
dfs = []
for llm_name, res_judge in judge_resutls.items():
    llm_name = llm_name.split("/")[-1]
    mean_col = f"{llm_name} (mean)"
    se_col = f"{llm_name} (SE)"
    res_judge = res_judge.rename(columns={"mean_judge_score": mean_col, "se_judge_score": se_col})[[mean_col]]
    dfs.append(res_judge)

df = pd.concat(dfs, axis=1)
df.head()

,Qwen3-32B (mean),Llama-3.1-8B-Instruct (mean)
field,,
dowody,0.833759,0.425818
inne_podstawy_prawne,0.373144,0.337033
miejsce_naruszenia,0.554942,0.394809
naruszenie_dobr_osobistych,0.768084,0.694644
naruszenie_media_spolecznosciowe,0.212038,0.134732


In [5]:
dfs = []
for llm_name, res_ngram in ngram_results.items():
    llm_name = llm_name.split("/")[-1]
    mean_col = f"{llm_name} (mean)"
    se_col = f"{llm_name} (SE)"
    res_ngram = res_ngram.rename(columns={"ngram_metric_value": mean_col})[[mean_col]]
    dfs.append(res_ngram)

df = pd.concat(dfs, axis=1)
df.head()

,,Qwen3-32B (mean),Llama-3.1-8B-Instruct (mean)
field,ngram_metric,,
dowody,f1,0.737483,0.365567
inne_podstawy_prawne,f1,0.043381,0.000777
miejsce_naruszenia,exact_match,0.558628,0.428096
naruszenie_dobr_osobistych,exact_match,0.768805,0.690172
naruszenie_media_spolecznosciowe,exact_match,0.201327,0.091616
